In [1]:
#蘋果日報

import requests
import json
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd
from lxml import etree
import json
import time,random,datetime,re
from tomorrow import threads
from pymongo import MongoClient
from datetime import datetime

In [2]:
#搜尋網址基本連結
ua = UserAgent()
headers = {"User-Agent":ua.random}

In [3]:
def GetAllLinks():
    
    links = []
    page = 1
    domin = "https://tw.news.appledaily.com"
    while True :
        try:
            page_url = "https://tw.news.appledaily.com/politics/realtime/" + str(page) 
            res = requests.get(page_url, headers = headers)
            soup = BeautifulSoup(res.text,"html.parser")
            for i in soup.find('div',{'class':'abdominis rlby clearmen'}).find_all("a",target="_blank"):
                links.append(domin + i.get('href')) 
            print("現在頁數：",page)
            page += 1

        except:
            break
            print(f'總頁數:{page}')
        time.sleep(random.randint(3,8))
    
    return links

In [4]:
#爬內文

def AppleCrawler(response):

    soup = BeautifulSoup(response.text)
    Url = soup.find('link',{'rel':'canonical'})['href']
    Title = soup.find('article',class_='ndArticle_leftColumn').find('h1').text.replace('\u3000','')
    CreateTime = datetime.strptime(soup.find('div',{'class':'ndArticle_creat'}).text.replace('出版時間：',''),'%Y/%m/%d %H:%M')
    content_list = []
    any(content_list.append(i.text) for i in soup.find('div',class_='ndArticle_margin').find_all('p'))
    Content = "".join(content_list)
    Tag = soup.find('meta',{'name':'keywords'})['content'].split(',')

    ndf = pd.DataFrame(data = [{'Source':'News',
                                'Site':'蘋果日報',
                                'Url':Url,
                                'Title':Title,
                                'CreateTime':CreateTime,
                                'Author':'無作者',
                                'Content':Content,
                                'Tag':Tag}],
                                columns = ['Source','Site','Url','CreateTime','Author','Title','Content','Tag'])
    time.sleep(random.randint(5,8))
    return ndf

In [5]:
#測試抓取一篇文章並存為是否成功

url = "https://tw.news.appledaily.com/politics/realtime/20191123/1667642/"
res = requests.get(url,headers=headers)
AppleCrawler(res)


,Source,Site,Url,CreateTime,Author,Title,Content,Tag
0,News,蘋果日報,https://tw.appledaily.com/new/realtime/2019112...,2019-11-23 10:45:00,無作者,李佳芬發燒缺席汪志冰造勢李四川代打站台,國民黨總統參選人韓國瑜的太太李佳芬原預定今（23日）出席黨內立委參選人汪志冰競選造勢活動，卻...,"[韓國瑜, 李佳芬, 高美蘭]"


In [6]:
# 多線程爬蟲定義
@threads(100)
def MultiThread_Crawl(url):
    try:
        return requests.get(url, headers=headers)
    except:
        pass

In [7]:
def CrawlAllNews(links):
    # 爬取全部連結的新聞內容
    print('There are {} links in pages.'.format(len(links)))
    
    # 多線程爬蟲
    responses = [MultiThread_Crawl(link) for link in links]

    # 整理成DataFrame
    allData = []
    for response in responses:
        try:
            ndf = AppleCrawler(response)
            allData.append(ndf)
        except:
            pass
    df = pd.concat(allData, ignore_index=True)
    print('There are {} News in DataFrame.'.format(len(df)))
    return df


In [8]:
%%time
links = GetAllLinks() 
df = CrawlAllNews(links)

現在頁數： 1
現在頁數： 2
現在頁數： 3
現在頁數： 4
現在頁數： 5
現在頁數： 6
現在頁數： 7
現在頁數： 8
現在頁數： 9
現在頁數： 10
現在頁數： 11
現在頁數： 12
現在頁數： 13
現在頁數： 14
現在頁數： 15
There are 421 links in pages.
There are 79 News in DataFrame.


In [13]:
def dataframe_to_mongo_test(select_df):
    user_name = "test"
    password = "pn123456"
    connect_IP = "ntcpn108.ddns.net"
    DB_name = "test"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.test   #DB_name
    collection = db.test_news  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")


In [ ]:
def dataframe_to_mongo(select_df):
    user_name = "pol"
    password = "pn123456"
    connect_IP = "192.168.1.126"
    DB_name = "political"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.political   #DB_name
    collection = db.news_pol  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")
        

In [ ]:
%%time
links = GetAllLinks()

allData = []
seccess_count = 0
error_count = 0
for link in links:
    try:
        res = requests.get(link,headers=headers)
        ndf = AppleCrawler(res)
        allData.append(ndf)
        seccess_count += 1
    
    except:
        error_count += 1
        pass
df = pd.concat(allData, ignore_index=True)
dataframe_to_mongo_test(df)



現在頁數： 1
現在頁數： 2
現在頁數： 3
現在頁數： 4
現在頁數： 5
現在頁數： 6
現在頁數： 7
現在頁數： 8
現在頁數： 9
現在頁數： 10
現在頁數： 11
現在頁數： 12
現在頁數： 13
